In [1]:
import pandas as pd
import urllib
from scrape_flight_crl import parse_flights, get_arrivals, get_departures, URL

In [2]:
df = parse_flights()
arrivals, departures = get_arrivals(df), get_departures(df)

In [19]:
arrivals.tail()

Flightnr. Depart Arrive      Start        End   Mon.   Tue.  \
connection    #                                                               
Vilnius->CRL  1    FR6459  21:10  22:45 2017-12-28 2017-12-28  False  False   
              2    FR6459  21:10  22:45 2018-04-01 2018-03-22  False  False   
Zaragoza->CRL 0    FR1383  10:25  12:30 2017-12-19 2017-12-19  False   True   
              1    FR1383  10:25  12:30 2018-02-01 2018-02-01  False   True   
              2    FR1383  19:25  21:30 2017-01-11 2018-03-24  False  False   

                  Wed.   Thu.   Fri.   Sat.   Sun.  
connection    #                                     
Vilnius->CRL  1  False   True  False  False  False  
              2  False   True  False  False   True  
Zaragoza->CRL 0  False  False  False  False  False  
              1  False  False  False  False  False  
              2   True  False  False   True  False

In [4]:
arrivals.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 495 entries, (Agadir->CRL, 0) to (Zaragoza->CRL, 2)
Data columns (total 12 columns):
Flightnr.    495 non-null object
Depart       495 non-null object
Arrive       495 non-null object
Start        495 non-null datetime64[ns]
End          495 non-null datetime64[ns]
Mon.         495 non-null bool
Tue.         495 non-null object
Wed.         495 non-null object
Thu.         495 non-null object
Fri.         495 non-null object
Sat.         495 non-null object
Sun.         495 non-null object
dtypes: bool(1), datetime64[ns](2), object(9)
memory usage: 46.1+ KB


In [5]:
arrivals_flat = arrivals.reset_index()
arrivals_flat.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,Mon.,Tue.,Wed.,Thu.,Fri.,Sat.,Sun.
490,Vilnius->CRL,1,FR6459,21:10,22:45,2017-12-28,2017-12-28,False,False,False,True,False,False,False
491,Vilnius->CRL,2,FR6459,21:10,22:45,2018-04-01,2018-03-22,False,False,False,True,False,False,True
492,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,False,True,False,False,False,False,False
493,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,False,True,False,False,False,False,False
494,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,False,False,True,False,False,True,False


In [6]:
arrivals_flat.columns[7:]

Index(['Mon.', 'Tue.', 'Wed.', 'Thu.', 'Fri.', 'Sat.', 'Sun.'], dtype='object')

Rename columns to make easy the sorting later

In [7]:
d = dict(zip(arrivals_flat.columns[7:], range(7)))
d

{'Fri.': 4, 'Mon.': 0, 'Sat.': 5, 'Sun.': 6, 'Thu.': 3, 'Tue.': 1, 'Wed.': 2}

In [8]:
arrivals_flat.rename(columns=d, inplace=True)
arrivals_flat.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,0,1,2,3,4,5,6
490,Vilnius->CRL,1,FR6459,21:10,22:45,2017-12-28,2017-12-28,False,False,False,True,False,False,False
491,Vilnius->CRL,2,FR6459,21:10,22:45,2018-04-01,2018-03-22,False,False,False,True,False,False,True
492,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,False,True,False,False,False,False,False
493,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,False,True,False,False,False,False,False
494,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,False,False,True,False,False,True,False


In [9]:
wd = arrivals_flat.melt(id_vars=arrivals_flat.columns[:7], value_vars=arrivals_flat.columns[7:], var_name="weekday")

In [10]:
wd.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,weekday,value
3460,Vilnius->CRL,1,FR6459,21:10,22:45,2017-12-28,2017-12-28,6,False
3461,Vilnius->CRL,2,FR6459,21:10,22:45,2018-04-01,2018-03-22,6,True
3462,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,6,False
3463,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,6,False
3464,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,6,False


In [11]:
wd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3465 entries, 0 to 3464
Data columns (total 9 columns):
connection    3465 non-null object
#             3465 non-null int64
Flightnr.     3465 non-null object
Depart        3465 non-null object
Arrive        3465 non-null object
Start         3465 non-null datetime64[ns]
End           3465 non-null datetime64[ns]
weekday       3465 non-null object
value         3465 non-null object
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 243.7+ KB


In [12]:
wd[wd.loc[:,"value"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 3461
Data columns (total 9 columns):
connection    1100 non-null object
#             1100 non-null int64
Flightnr.     1100 non-null object
Depart        1100 non-null object
Arrive        1100 non-null object
Start         1100 non-null datetime64[ns]
End           1100 non-null datetime64[ns]
weekday       1100 non-null object
value         1100 non-null object
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 85.9+ KB


In [13]:
wd_true = wd[wd.loc[:,"value"]].drop(columns="value")

In [14]:
wd_true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 3461
Data columns (total 8 columns):
connection    1100 non-null object
#             1100 non-null int64
Flightnr.     1100 non-null object
Depart        1100 non-null object
Arrive        1100 non-null object
Start         1100 non-null datetime64[ns]
End           1100 non-null datetime64[ns]
weekday       1100 non-null object
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 77.3+ KB


In [24]:
wd_true_sorted = wd_true.sort_values(["connection", "weekday"]).reset_index(drop=True)
wd_true_sorted.tail()

,connection,#,Flightnr.,Depart,Arrive,Start,End,weekday
1095,Warsaw->CRL,6,W61381,18:40,20:40,2017-10-29,2017-12-15,6
1096,Zaragoza->CRL,0,FR1383,10:25,12:30,2017-12-19,2017-12-19,1
1097,Zaragoza->CRL,1,FR1383,10:25,12:30,2018-02-01,2018-02-01,1
1098,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,2
1099,Zaragoza->CRL,2,FR1383,19:25,21:30,2017-01-11,2018-03-24,5


In [26]:
wd_true_sorted['weekday'] = wd_true_sorted['weekday'].replace({v: k for k, v in d.items()})

In [31]:
arrivals_weekdays = wd_true_sorted.set_index(["connection", "weekday"])
arrivals_weekdays

# Flightnr. Depart Arrive      Start        End
connection    weekday                                                  
Agadir->CRL   Mon.      0    FR8173  09:45  14:35 2017-10-30 2017-12-22
              Mon.      4    FR8173  09:45  14:35 2018-08-01 2018-03-23
              Mon.      5    FR8173  11:15  16:05 2018-01-01 2018-01-01
              Tue.      1    FR8173  09:45  14:35 2017-12-19 2017-12-19
              Tue.      3    FR8173  09:45  14:35 2018-02-01 2018-02-01
              Wed.      0    FR8173  09:45  14:35 2017-10-30 2017-12-22
              Wed.      2    FR8173  09:45  14:35 2017-12-27 2018-05-01
              Wed.      4    FR8173  09:45  14:35 2018-08-01 2018-03-23
              Thu.      6    FR8173  18:05  22:55 2017-12-21 2018-04-01
              Fri.      0    FR8173  09:45  14:35 2017-10-30 2017-12-22
              Fri.      2    FR8173  09:45  14:35 2017-12-27 2018-05-01
              Fri.      4    FR8173  09:45  14:35 2018-08-01 2018-03-23
Algiers->CRL  Thu.      0     TB854  19:00  21:20 2017-02-11 2018-03-22
              Sun.      1    TB1331  19:00  21:20 2017-12-11 2017-11-19
              Sun.      2    TB1332  19:00  21:20 2017-11-26 2017-11-26
              Sun.      3    TB1385  19:00  21:20 2017-03-12 2018-01-14
              Sun.      4    TB1385  19:00  21:20 2018-01-21 2018-11-02
              Sun.      5    TB1385  19:00  21:20 2018-02-18 2018-03-18
Alicante->CRL Mon.      0     TB223  10:35  13:10 2017-10-30 2018-03-19
              Mon.     10    FR9052  20:00  22:35 2017-06-11 2017-12-23
              Mon.     11    FR9052  20:00  22:35 2017-12-29 2018-06-01
              Mon.     12    FR9052  20:00  22:35 2018-08-01 2018-03-24
              Tue.      4    FR9052  17:45  20:20 2017-12-19 2018-02-01
              Wed.      2     TB248  17:35  20:10 2017-01-11 2018-03-21
              Wed.      3    FR9052  17:45  20:20 2017-01-11 2018-03-22
              Thu.      3    FR9052  17:45  20:20 2017-01-11 2018-03-22
              Fri.      6    FR9052  19:45  22:20 2017-10-11 2017-12-22
              Fri.      7    FR9052  19:45  22:20 2018-12-01 2018-02-23
              Fri.      8    FR9052  19:45  22:20 2018-02-03 2018-02-03
              Fri.      9    FR9052  19:45  22:20 2018-09-03 2018-03-23
...                    ..       ...    ...    ...        ...        ...
Warsaw->CRL   Thu.      2    W61381  12:20  14:30 2018-01-16 2018-03-20
              Thu.      3    W61381  18:30  19:40 2017-12-16 2017-12-23
              Thu.      4    W61381  18:30  20:40 2017-12-27 2018-06-01
              Thu.      5    W61381  18:30  20:40 2018-07-01 2018-03-23
              Thu.      6    W61381  18:40  20:40 2017-10-29 2017-12-15
              Fri.      0    W61381  11:40  13:50 2017-12-19 2018-09-01
              Fri.      1    W61381  12:20  14:30 2017-10-31 2017-12-12
              Fri.      2    W61381  12:20  14:30 2018-01-16 2018-03-20
              Fri.      3    W61381  18:30  19:40 2017-12-16 2017-12-23
              Fri.      4    W61381  18:30  20:40 2017-12-27 2018-06-01
              Fri.      5    W61381  18:30  20:40 2018-07-01 2018-03-23
              Fri.      6    W61381  18:40  20:40 2017-10-29 2017-12-15
              Sat.      0    W61381  11:40  13:50 2017-12-19 2018-09-01
              Sat.      1    W61381  12:20  14:30 2017-10-31 2017-12-12
              Sat.      2    W61381  12:20  14:30 2018-01-16 2018-03-20
              Sat.      3    W61381  18:30  19:40 2017-12-16 2017-12-23
              Sat.      4    W61381  18:30  20:40 2017-12-27 2018-06-01
              Sat.      5    W61381  18:30  20:40 2018-07-01 2018-03-23
              Sat.      6    W61381  18:40  20:40 2017-10-29 2017-12-15
              Sun.      0    W61381  11:40  13:50 2017-12-19 2018-09-01
              Sun.      1    W61381  12:20  14:30 2017-10-31 2017-12-12
              Sun.      2    W61381  12:20  14:30 2018-01-16 2018-03-20
              Sun.      3    W61381  18:30  19:40 2017-12-16 201

In [42]:
arrivals_weekdays["Mon."]

KeyError: 'Mon.'